# 🚀 Início Rápido - Sistema de Previsão de Mercado Imobiliário

Este notebook demonstra como usar o **Sistema DSGE-RL** para fazer previsões do mercado imobiliário de Vitória/ES.

## O que você vai aprender:
- Como carregar dados de mercado
- Como executar o pipeline de previsão
- Como visualizar resultados
- Como interpretar o índice IDCI-VIX
- Como gerar previsões de 12 meses

**Design:** Gráficos minimalistas em preto e branco para relatórios profissionais

In [ ]:
# Importações
import sys
sys.path.append('../src')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Importar sistema
from pipeline import VitoriaForecastPipeline
from utils.plot_results import plot_all_results, print_summary

# Configuração de estilo minimalista
plt.style.use('grayscale')
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 10
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.alpha'] = 0.3

print("✓ Sistema importado com sucesso")
print(f"  Versão: Pipeline Vitória/ES Forecast v1.0")

## 1. Carregar Dados de Mercado

O sistema espera um DataFrame com séries temporais mensais. 

**Variáveis típicas:**
- `preco_m2`: Preço médio por m²
- `lancamentos`: Número de lançamentos
- `credito_imob`: Crédito imobiliário
- `emprego_construcao`: Emprego na construção civil
- `massa_salarial`: Massa salarial
- `pib_es`: PIB do Espírito Santo
- `selic`: Taxa Selic

In [ ]:
# ============================================================================
# OPÇÃO 1: Carregar seus dados (descomente e ajuste o caminho)
# ============================================================================
# df = pd.read_csv('../data/raw/dados_mercado.csv', index_col=0, parse_dates=True)
# df = pd.read_parquet('../data/raw/dados_mercado.parquet')

# ============================================================================
# OPÇÃO 2: Usar dados sintéticos para demonstração
# ============================================================================
print("Gerando dados sintéticos para demonstração...")
print("(Substitua por seus dados reais usando as linhas acima)\n")

np.random.seed(42)
dates = pd.date_range('2010-01-01', periods=120, freq='MS')

# Simula tendências e ciclos do mercado
trend = np.linspace(0, 2, 120)  # Tendência de crescimento
cycle = 0.5 * np.sin(2 * np.pi * np.arange(120) / 12)  # Ciclo anual

df = pd.DataFrame({
    'preco_m2': trend + cycle + 0.2 * np.random.randn(120).cumsum(),
    'lancamentos': 0.8 * trend + 0.3 * cycle + 0.15 * np.random.randn(120).cumsum(),
    'credito_imob': 0.9 * trend + 0.4 * cycle + 0.1 * np.random.randn(120).cumsum(),
    'emprego_construcao': 0.7 * trend + 0.2 * cycle + 0.2 * np.random.randn(120).cumsum(),
    'massa_salarial': 0.6 * trend + 0.1 * cycle + 0.15 * np.random.randn(120).cumsum(),
    'pib_es': 0.85 * trend + 0.25 * cycle + 0.1 * np.random.randn(120).cumsum(),
    'selic': -0.3 * trend + 0.1 * np.random.randn(120).cumsum(),
}, index=dates)

# Visualizar dados
print(f"📊 DADOS CARREGADOS")
print("="*60)
print(f"Período: {df.index[0].strftime('%Y-%m')} a {df.index[-1].strftime('%Y-%m')}")
print(f"Observações: {len(df)}")
print(f"Variáveis: {len(df.columns)}")
print(f"\nColunas: {', '.join(df.columns)}")

df.head()

In [ ]:
# Visualizar séries originais
fig, axes = plt.subplots(3, 3, figsize=(16, 10))
fig.suptitle('Séries Temporais do Mercado Imobiliário', fontsize=16, fontweight='bold')

for i, col in enumerate(df.columns):
    ax = axes[i // 3, i % 3]
    ax.plot(df.index, df[col], color='black', linewidth=2)
    ax.set_title(col.replace('_', ' ').title(), fontweight='bold')
    ax.set_xlabel('')
    ax.grid(True, alpha=0.3)
    ax.tick_params(axis='x', rotation=45)

# Remover eixos extras
for i in range(len(df.columns), 9):
    axes[i // 3, i % 3].axis('off')

plt.tight_layout()
plt.show()

print("✓ Séries carregadas e visualizadas")

## 2. Configurar e Executar Pipeline

O pipeline executa automaticamente:
1. **Pré-processamento**: Torna séries estacionárias
2. **Seleção de variáveis**: Identifica as mais relevantes via Granger
3. **Construção do IDCI-VIX**: Índice de confiança do mercado (0-10)
4. **Treinamento de modelos**: ARIMA, Ridge, Lasso, Random Forest, etc.
5. **Previsões**: Gera previsões de 12 meses
6. **Ensemble**: Combina modelos para melhor acurácia

In [ ]:
# Criar pipeline
pipeline = VitoriaForecastPipeline(
    max_vars=5,              # Número de variáveis a selecionar
    forecast_horizon=12,      # Previsão de 12 meses
    ar_order=2,              # Ordem autorregressiva
    verbose=True             # Mostrar progresso
)

print("\n✓ Pipeline configurado")
print(f"  • Variáveis máximas: {pipeline.max_vars}")
print(f"  • Horizonte de previsão: {pipeline.forecast_horizon} meses")
print(f"  • Ordem AR: {pipeline.ar_order}")

In [ ]:
# Executar pipeline completo
results = pipeline.run_full_pipeline(
    df,
    models_to_train=['arima', 'ridge', 'lasso', 'random_forest', 'quantile'],
    ensemble_method='weighted_avg'
)

## 3. Visualizar Resultados

### 3.1 Índice IDCI-VIX

O **IDCI-VIX** (Índice Dinâmico de Confiança Imobiliária - Vitória) é um indicador de 0 a 10 que resume o estado do mercado:
- **0-3**: Mercado pessimista
- **3-7**: Mercado neutro
- **7-10**: Mercado otimista

In [ ]:
# Visualizar IDCI-VIX
fig, ax = plt.subplots(1, 1, figsize=(14, 6))

ax.plot(results['idci_vix'].index, results['idci_vix'], 
        color='black', linewidth=2.5, label='IDCI-VIX')

# Adicionar zonas
ax.axhspan(0, 3, alpha=0.1, color='black', label='Pessimista')
ax.axhspan(3, 7, alpha=0.05, color='gray', label='Neutro')
ax.axhspan(7, 10, alpha=0.1, color='lightgray', label='Otimista')

# Valor atual
valor_atual = results['idci_vix'].iloc[-1]
ax.axhline(y=valor_atual, color='gray', linestyle='--', linewidth=2, 
           label=f'Atual: {valor_atual:.2f}')

ax.set_title('IDCI-VIX: Índice de Confiança do Mercado Imobiliário', 
             fontsize=14, fontweight='bold', pad=15)
ax.set_xlabel('Data', fontweight='bold')
ax.set_ylabel('IDCI-VIX (0-10)', fontweight='bold')
ax.set_ylim(0, 10)
ax.legend(loc='best')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Estatísticas
print("\n📊 ESTATÍSTICAS DO IDCI-VIX")
print("="*60)
print(f"Valor atual: {valor_atual:.2f}")
print(f"Média histórica: {results['idci_vix'].mean():.2f}")
print(f"Mínimo: {results['idci_vix'].min():.2f}")
print(f"Máximo: {results['idci_vix'].max():.2f}")
print(f"Desvio padrão: {results['idci_vix'].std():.2f}")

### 3.2 Previsões dos Modelos

In [ ]:
# Comparar previsões de todos os modelos
fig, ax = plt.subplots(1, 1, figsize=(14, 7))

# Histórico
ax.plot(results['idci_vix'].index, results['idci_vix'], 
        color='black', linewidth=2.5, label='Histórico', zorder=10)

# Gerar datas futuras para previsões
last_date = results['idci_vix'].index[-1]
future_dates = pd.date_range(last_date + pd.DateOffset(months=1), 
                             periods=pipeline.forecast_horizon, 
                             freq='MS')

# Previsões de cada modelo
colors = {'arima': 'darkgray', 'ridge': 'gray', 'lasso': 'lightgray', 
          'random_forest': 'silver', 'quantile': 'darkgray'}
linestyles = {'arima': '--', 'ridge': '-.', 'lasso': ':', 
              'random_forest': '--', 'quantile': '-.'}

for model_name, forecast in results['forecasts'].items():
    if model_name.endswith('_quantiles'):
        continue
    
    if 'forecast' in forecast.columns:
        ax.plot(future_dates, forecast['forecast'], 
                color=colors.get(model_name, 'gray'),
                linestyle=linestyles.get(model_name, '-'),
                linewidth=1.5, alpha=0.7,
                label=f'{model_name.upper()}')

# Ensemble (destaque)
ax.plot(future_dates, results['ensemble']['forecast'], 
        color='black', linewidth=3, linestyle='-',
        marker='o', markersize=6, markerfacecolor='white',
        label='ENSEMBLE', zorder=11)

# Linha vertical separando histórico de previsão
ax.axvline(x=last_date, color='black', linestyle=':', linewidth=1.5, alpha=0.5)
ax.text(last_date, ax.get_ylim()[1]*0.95, ' Previsão →', 
        fontsize=10, fontweight='bold', va='top')

ax.set_title('Previsões de 12 Meses - Comparação de Modelos', 
             fontsize=14, fontweight='bold', pad=15)
ax.set_xlabel('Data', fontweight='bold')
ax.set_ylabel('IDCI-VIX', fontweight='bold')
ax.legend(loc='best', ncol=2)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✓ Previsões visualizadas")

### 3.3 Variáveis Selecionadas

O sistema selecionou automaticamente as variáveis mais importantes via teste de Granger:

In [ ]:
print("\n🎯 VARIÁVEIS SELECIONADAS")
print("="*60)
for i, var in enumerate(results['selected_vars'], 1):
    print(f"  {i}. {var}")

# Visualizar contribuição
fig, ax = plt.subplots(1, 1, figsize=(12, 5))

# Calcular correlação com IDCI-VIX
correlacoes = []
for var in results['selected_vars']:
    corr = results['data_stationary'][var].corr(results['idci_vix'])
    correlacoes.append(abs(corr))

# Plot
bars = ax.barh(range(len(results['selected_vars'])), correlacoes,
               color='black', alpha=0.7, edgecolor='white', linewidth=2)

ax.set_yticks(range(len(results['selected_vars'])))
ax.set_yticklabels(results['selected_vars'])
ax.invert_yaxis()
ax.set_xlabel('Correlação Absoluta com IDCI-VIX', fontweight='bold')
ax.set_title('Importância das Variáveis Selecionadas', fontweight='bold', pad=15)
ax.grid(True, alpha=0.3, axis='x')

# Adicionar valores
for i, (bar, corr) in enumerate(zip(bars, correlacoes)):
    ax.text(corr, i, f' {corr:.3f}', va='center', fontweight='bold')

plt.tight_layout()
plt.show()

## 4. Resumo Executivo

In [ ]:
print("\n" + "="*80)
print("📊 RESUMO EXECUTIVO - PREVISÃO DO MERCADO IMOBILIÁRIO")
print("="*80)

print("\n🎯 INDICADOR ATUAL")
print("-"*80)
idci_atual = results['idci_vix'].iloc[-1]
print(f"IDCI-VIX Atual: {idci_atual:.2f}")

if idci_atual < 3:
    print("Status: 🔴 PESSIMISTA - Mercado em retração")
elif idci_atual < 7:
    print("Status: 🟡 NEUTRO - Mercado estável")
else:
    print("Status: 🟢 OTIMISTA - Mercado em expansão")

print("\n📈 PREVISÕES")
print("-"*80)
ensemble_12m = results['ensemble']['forecast'].iloc[0]
variacao_12m = ((ensemble_12m - idci_atual) / idci_atual) * 100

print(f"Previsão 12 meses (Ensemble): {ensemble_12m:.2f}")
print(f"Variação esperada: {variacao_12m:+.1f}%")

if variacao_12m > 5:
    print("Tendência: ↗️ ALTA - Expectativa de crescimento")
elif variacao_12m < -5:
    print("Tendência: ↘️ BAIXA - Expectativa de retração")
else:
    print("Tendência: → ESTÁVEL - Manutenção do patamar")

print("\n🎲 MODELOS")
print("-"*80)
print(f"Modelos treinados: {len(results['models'])}")
print(f"Variáveis utilizadas: {len(results['selected_vars'])}")
print(f"Período analisado: {len(results['idci_vix'])} meses")

print("\n💡 RECOMENDAÇÕES")
print("-"*80)
if idci_atual >= 7 and variacao_12m > 0:
    print("• Momento favorável para lançamentos")
    print("• Demanda aquecida")
    print("• Considerar estratégias de precificação premium")
elif idci_atual <= 3 and variacao_12m < 0:
    print("• Cautela em novos investimentos")
    print("• Foco em liquidez e VGV")
    print("• Revisar estratégia de marketing e vendas")
else:
    print("• Monitorar indicadores mensalmente")
    print("• Manter estratégia equilibrada")
    print("• Preparar cenários alternativos")

print("\n" + "="*80)

## 5. Salvar Resultados

In [ ]:
# Criar diretório se não existir
import os
os.makedirs('../data/processed', exist_ok=True)

# Salvar IDCI-VIX
results['idci_vix'].to_csv('../data/processed/idci_vix.csv', header=True)

# Salvar previsões ensemble
forecast_df = pd.DataFrame({
    'data': future_dates,
    'previsao': results['ensemble']['forecast'].values
})
forecast_df.to_csv('../data/processed/previsao_ensemble_12m.csv', index=False)

# Salvar todas as previsões
all_forecasts = pd.DataFrame(index=future_dates)
for model_name, forecast in results['forecasts'].items():
    if not model_name.endswith('_quantiles') and 'forecast' in forecast.columns:
        all_forecasts[model_name] = forecast['forecast'].values
all_forecasts['ensemble'] = results['ensemble']['forecast'].values
all_forecasts.to_csv('../data/processed/previsoes_todos_modelos.csv')

print("✓ Resultados salvos em data/processed/")
print("  • idci_vix.csv - Série histórica do índice")
print("  • previsao_ensemble_12m.csv - Previsão combinada")
print("  • previsoes_todos_modelos.csv - Todas as previsões")

## 📚 Próximos Passos

Agora que você sabe usar o sistema básico, explore:

1. **02_previsao_precos.ipynb** - Previsão detalhada de preços por m²
2. **03_analise_cenarios.ipynb** - Simulação de diferentes cenários econômicos
3. **04_otimizacao_parametros.ipynb** - Comparação e otimização de modelos

---

## 💡 Dicas

- **Dados reais**: Substitua os dados sintéticos pelos seus dados históricos
- **Atualização**: Execute mensalmente com dados novos
- **Validação**: Compare previsões passadas com valores realizados
- **Customização**: Ajuste parâmetros do pipeline conforme necessário